### Hybrid Model of Quantum Transfer Learning with ResNet-18 and Basic Entangler Layers to Classify Face Images with a COVID-19 Protective Mask
- Autor: Soto Paredes, Christian


In [ ]:
#Pennylane
!pip install git+https://github.com/PennyLaneAI/pennylane.git # instalacion pennylane version de desarrollo (no estable) porque maneja mejor la GPU
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import AngleEmbedding, BasicEntanglerLayers, SimplifiedTwoDesign, StronglyEntanglingLayers
from pennylane.operation import Operation, AnyWires
from pennylane.init import strong_ent_layers_uniform, strong_ent_layers_normal

#Qiskit
#!pip install qiskit ipywidgets
#!pip install pylatexenc
#!pip install pennylane-qiskit
#!pip install qiskit-aer-gpu
#from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
#from qiskit.visualization import plot_state_city
#from numpy import pi

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

# utiles
import datetime
import time
from time import process_time
import os
import copy
import matplotlib.pyplot as plt

#deshabilitar warnings
import warnings
warnings.filterwarnings("ignore")


print("\nVersion de PyTorch: ", torch.__version__)
print("Version de Torchvision: ", torchvision.__version__)

####################################
# Accesar a mi drive personal
# -----------------------------------
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
####################################
# inicializacion de hiperparámetros
#-----------------------------------

#ruta donde se guardan los modelos
path_modelos = "/content/drive/MyDrive/modelos/"

# tamaño de la imagen
global tam_imagen

# Numero de muestras para cada entrenamiento, tamaño del batch para el entrenamiento (Cambiar dependiendo de cuanta memoria tenemos)
tam_lote = 8

# Flag para feature extracting. Cuando es False, hacemos finetune al modelo completo, cuando es True  solo actualizamos los parametros de la capa final
feature_extract = False

# Numero de clases/salidas 
num_clases = 3

# Vemos si el GPU esta habilitado
procesador = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
           
# Numero de epocas para el entrenamiento (para alexnet se recomienda 15)
num_epocas = 10

#flag para saber si el modelo se ha cargado a partir de un modelo guardado
#flg_cargo_modelo = False

#numero de qubits
num_qubits = 4

# numero de capas cuanticas, re recomienda entre 6 y 15 es recomendado.
num_capas_q = 10

# Tasa de propagacion aleatoria de los pesos cuanticos iniciales
q_delta = 0.01   

# Numero de Shots, para un nivel de confianza del 95%, z=-1.96, por lo tanto se necesita 385 
shots = 385

# Eje sobre el que se realiza la rotacion
rotacion = "Y"

# Que circuito se va elegir para el diseño de la red cuantica
circuito_op = "circuitoPersonalizado"

modelo_ft = None
optimizador_ft = None


In [ ]:


################################################
# Definicion de la red neuronal Cuantica
# ---------------------------------------------
#
# En Pennylane, la computacion cuantica es representada como objetos compuestos de nodos cuanticos
# un nodo cuantico es aquel cuyo uso es agarrar un circuito cuantico y relacionarlo con un dispositivo que ejecute dicho circuito

# metodos de diferenciacion para simuladores: best, backprop, adjoint, reversible, por defecto el best es usado
# para hardware real usar: best, parameter-shift, finite-diff
metodo_dif = "reversible" #esta funcionando bien 
#metodo_dif = "best" 

# Para ejecutar y luego optimizar un circuito cuantico, se necesita un dispositivo que no es otra cosa que una instancia de la clase Device, la cual nos permite indicar
# en que hardware ejecutar el circuito puede ser en un simulador local o en la nube o en un hardware real, **default.qubit** es el simulador local por defecto de pennylane
# el numero de **shots** indica cuantas veces el circuito deberia ser evaluado o sampleado para estimar cantidades estadisticas, shots=None indica medir estadisticas exactas

dispositivo = qml.device('default.qubit', wires=num_qubits )  

#from qiskit import IBMQ
#IBMQ.save_account("fe21d617766fc331f4c475c6b33989785e9a53da0f786f8782ba71a5ba1ae98604e108684ca9d1c705f584323353b98eefb52bb436abbb1b7baf9e573e264be0")
#IBMQ.providers()    # List all available providers
#provider = IBMQ.load_account()
##dispositivo = qml.device('qiskit.ibmq', wires=num_qubits, backend='ibmq_lima')
#dispositivo = qml.device('qiskit.ibmq', wires=num_qubits, backend='ibmq_qasm_simulator', shots=shots )
#dispositivo.capabilities()['backend']


# funcion cuantica que coloca puertas de Hadamard en todos los wires
def capa_H(num_qubits):
  for i in range(num_qubits):
    qml.Hadamard(wires = i) #wires indica en que qubit se encuentra 0,1,2,3

def capa_R(rotacion, w):    
  for i, valor in enumerate(w): # el  valor de la rotacion (RX,RY,RZ) para la primera capa recibe las caracteristicas de entrada y para las capas subsecuentes recibe los pesos
    rotacion(valor, wires=i)

# circuito con el uso de BasicEntanglerLayers
# -----------------------------------------------------
@qml.qnode(dispositivo, interface="torch", diff_method = metodo_dif) #con pennylane
#@qml.qnode(dispositivo, interface="torch")    # con qiskit
def circuitoBasicEntangler(entrada_caracteristicas, pesos):
  # redimensiona los pesos
  pesos = pesos.reshape(num_capas_q, num_qubits)
  # preparacion
  capa_H(num_qubits)  
  # 1. capa de entrada  
  AngleEmbedding(features=entrada_caracteristicas, wires=list(range(num_qubits)), rotation="Y")  
  # 2. capas de circuitos variacionales cuanticos
  BasicEntanglerLayers(weights=pesos, wires=list(range(num_qubits)), rotation=qml.RY)  
  # 3. capa de medidas
  salidas = [qml.expval(qml.PauliZ(wires=i)) for i in range(num_qubits)]
  return tuple(salidas)

# circuito con el uso de StronglyEntanglingLayers
# -----------------------------------------------------
@qml.qnode(dispositivo, interface="torch", diff_method = metodo_dif) #con pennylane
#@qml.qnode(dispositivo, interface="torch")    # con qiskit
def circuitoStronglyEntanglingLayers(entrada_caracteristicas, pesos):
  # redimensiona los pesos 
  #pesos = pesos.reshape(num_capas_q, num_qubits)

  # preparacion
  capa_H(num_qubits)  
  # 1. capa de entrada
  AngleEmbedding(features=entrada_caracteristicas, wires=list(range(num_qubits)), rotation="Y")
  # 2. capas de circuitos variacionales cuanticos
  StronglyEntanglingLayers(pesos, list(range(num_qubits)))
  # 3. capa de medidas
  salidas = [qml.expval(qml.PauliZ(wires=i)) for i in range(num_qubits)]
  return tuple(salidas)


# circuito personalizado de [2]
#----------------------------------------------------

# funcion cuantica que establece el entrelazamiento de los wires con puertas CNOT
def relaciones_CNOT(nqubits):
    # Bucle sobre indices pares: i=0,2,...N-2 
    # para 4 qbits: i=0,2
    for i in range(0, nqubits - 1, 2):
        qml.CNOT(wires=[i, i + 1])
    # Bucle sobre indices impares:  i=1,3,...N-3           
    # para 4 qbits: i=1
    for i in range(1, nqubits - 1, 2):  
        qml.CNOT(wires=[i, i + 1])

@qml.qnode(dispositivo, interface="torch", diff_method = metodo_dif) #con pennylane
#@qml.qnode(dispositivo, interface="torch")    # con qiskit
def circuitoPersonalizado(entrada_caracteristicas, pesos):
  # redimensiona los pesos a la forma num_capas_q x num_qubits
  pesos = pesos.reshape(num_capas_q, num_qubits)
  #preparacion
  capa_H(num_qubits)
  # 1. capa de entrada
  AngleEmbedding(features=entrada_caracteristicas, wires=list(range(num_qubits)), rotation=rotacion)
  # 2. capas de circuitos variacionales cuanticos  
  for i in range(num_capas_q):
    relaciones_CNOT(num_qubits)     
    #pesos
    AngleEmbedding(features=pesos[i], wires=list(range(num_qubits)), rotation=rotacion)
  # 3. capa de medidas
  return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

# Circuito Personalizado Christian
#----------------------------------------------------
def relaciones_CNOT_2(nqubits):
  for i in range(0, nqubits - 1,2):
    qml.CNOT(wires=[i, i + 1])
    if (i <= nqubits-3):
      qml.CNOT(wires=[i+1, i + 2])  
  if (nqubits>2):
    qml.CNOT(wires=[num_qubits-1, 0])
        

@qml.qnode(dispositivo, interface="torch", diff_method = metodo_dif) #con pennylane
#@qml.qnode(dispositivo, interface="torch")    # con qiskit
def circuitoPersonalizado2(entrada_caracteristicas, pesos):
  # redimensiona los pesos a la forma num_capas_q x num_qubits
  pesos = pesos.reshape(num_capas_q, num_qubits)
  #preparacion
  capa_H(num_qubits)
  # 1. capa de entrada
  #    utilizar una puerta de rotacion y girar tanto angulo como cantidad indique nuestros atributos de entrada
  AngleEmbedding(features=entrada_caracteristicas, wires=list(range(num_qubits)), rotation="Y") 
  # 2. capas de circuitos variacionales cuanticos  
  for i in range(num_capas_q):
    relaciones_CNOT_2(num_qubits)    
    # pesos
    # utilizar una puerta de rotacion y girar la cantidad que indique nuestros pesos
    AngleEmbedding(features=pesos[i], wires=list(range(num_qubits)), rotation="Y")
  # 3. capa de medidas
  return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

# Clase que define la red neuronal cuantica
# creamos una nueva clase de python que hereda el modulo nn.Module (nn.Module es una clase base para todos los modulos de redes neuronales de pytorch)
# nn.Linear crea una capa completamente conectada, aplica una transformacion lineal a los datos de entrada de la forma y = xA**T + b
# **nn.Linear(in_features=num_ftrs, out_features=num_qubits, bias=True))** 
# cuando entrenamos una red neuronal clasica se puede decir que estamos intentando llegar a nuestra funcion objetivo a traves de series de polinomios, 
# con una red cuantica estamos llegando a la funcion objetivo a traves de series de fourier es decir con los senos y los cosenos

class RedCuantica(nn.Module):
    
  def __init__(self, num_ftrs):
    # Todas las capas de la red son creadas en el constructor, todas las capas son lineales es decir una red "totalmente conectada",  que aplica una traducción lineal 
    # a todas las entradas (los valores en la capa se inicializan aleatoriamente)    
    super().__init__()
    self.num_ftrs = num_ftrs   # numero de caracteristicas 512 para resnet y squeezenet, 1024 densenet, 2048 inception    
    self.pre_net = nn.Linear(self.num_ftrs , num_qubits)        
    self.pesos = nn.Parameter(q_delta * torch.randn(num_capas_q  * num_qubits)) # para usar con BasicEntanglerLayers y el Personalizado
    #self.pesos = nn.Parameter(torch.tensor(q_delta * strong_ent_layers_uniform(n_layers=num_capas_q, n_wires=num_qubits)) ) # para usar en StronglyEntanglingLayers
    self.post_net = nn.Linear(num_qubits, num_clases) #la red finaliza con 3 clases/salidas
              
  def forward(self, input_features):    
    # establecemos la relacion entre capas en el metodo forward, forward nos muestra cómo fluye una imagen a través de la red, primero convertimos el tensor de la 
    # imagen en una forma que la primera capa pueda entender, aplanamos el tensor para que tenga la forma de la primera capa lineal
    # input_features: caracteristicas de las imagenes de entrada resnet 450x512
    #print("num_ftrs: ", self.num_ftrs)
    
    pre_out = self.pre_net(input_features) 
    # funcion de activacion     
    q_in = torch.tanh(pre_out) * np.pi / 2.0    # multiplicamos por pi/2 porque solo necesitamos de -1 a 1 (el tanh va desde -pi/2 hasta pi/2)
    

    # Aplicamos el circuito cuantico a cada elemento del bloque (batch), y añadimos a q_out
    q_out = torch.Tensor(0, num_qubits)
    q_out = q_out.to(procesador)

    for caracteristica in q_in:
      if (circuito_op == "circuitoPersonalizado"):
        q_out_elem = circuitoPersonalizado(caracteristica, self.pesos).float().unsqueeze(0) # para usar con el circuito Personalizado
      elif (circuito_op == "circuitoBasicEntangler"):
        q_out_elem = circuitoBasicEntangler(caracteristica, self.pesos).float().unsqueeze(0) # para usar con BasicEntanglerLayers
      elif (circuito_op == "circuitoStronglyEntanglingLayers"):
        q_out_elem = circuitoStronglyEntanglingLayers(caracteristica, self.pesos).float().unsqueeze(0) # para usar con StronglyEntanglingLayers
      elif (circuito_op == "circuitoPersonalizado2"):
        q_out_elem = circuitoPersonalizado2(caracteristica, self.pesos).float().unsqueeze(0) # para usar con el circuito Personalizado Christian
      else:
        print("Nombre de circuito incorrecto, saliendo...")
        exit()
      q_out_elem = q_out_elem.to(procesador)
      q_out = torch.cat((q_out, q_out_elem))
    return self.post_net(q_out)

#grafica el circuito cuantico

#circuitoPersonalizado
print("circuitoPersonalizado:\n")
#print(circuitoPersonalizado([0.1,0.2,0.3,0.4], nn.Parameter(q_delta * torch.randn(num_capas_q * num_qubits))))
grafico1 = qml.draw(circuitoPersonalizado)
print(grafico1([0.1,0.2,0.3,0.4], nn.Parameter(q_delta * torch.randn(num_capas_q * num_qubits))))

#circuitoBasicEntangler
print("circuitoBasicEntangler:\n")
#print(circuitoBasicEntangler([0.1,0.2,0.3,0.4], nn.Parameter(q_delta * torch.randn(num_capas_q * num_qubits))))
grafico2 = qml.draw(circuitoBasicEntangler)
print(grafico2([0.1,0.2,0.3,0.4], nn.Parameter(q_delta * torch.randn(num_capas_q * num_qubits))))

#circuitoStronglyEntanglingLayers
print("circuitoStronglyEntanglingLayers:\n")
pesos_ini = strong_ent_layers_uniform(n_layers=num_capas_q, n_wires=num_qubits)
_pesos_ini = strong_ent_layers_normal(n_layers=num_capas_q, n_wires=num_qubits)
_pesos_ini = torch.tensor(pesos_ini, requires_grad=True)
#print(circuitoStronglyEntanglingLayers([0.1,0.2,0.3,0.4], _pesos_ini))
grafico3 = qml.draw(circuitoStronglyEntanglingLayers)
print(grafico3([0.1,0.2,0.3,0.4], pesos_ini))

print("circuitoPersonalizado Christian:\n")
#print(circuitoPersonalizado2([0.1,0.2,0.3,0.4], nn.Parameter(q_delta * torch.randn(num_capas_q * num_qubits))))
grafico4 = qml.draw(circuitoPersonalizado2)
print(grafico4([0.1,0.2,0.3,0.4], nn.Parameter(q_delta * torch.randn(num_capas_q * num_qubits))))

print(f"Procesador: {procesador}\nMétodo de diferenciacion: {metodo_dif}\nCircuito: {circuito_op}"
      f"\nNumero capas cuanticas: {num_capas_q}\nNumero de epocas: {num_epocas}\nNumero de qubits: {num_qubits}")


circuitoPersonalizado:

 0: ──H──RY(0.1)──╭C───RY(-0.00255)────────────────╭C───RY(-0.0019)──────────────╭C───RY(-0.000353)───────────────╭C───RY(-0.003)─────────────────╭C───RY(0.0123)──────────────────╭C───RY(-0.00212)───────────────╭C───RY(0.0053)──────────────────╭C───RY(-0.0161)────────────────╭C───RY(0.0015)───────────────╭C───RY(-0.00742)───────────────┤ ⟨Z⟩ 
 1: ──H──RY(0.2)──╰X──╭C─────────────RY(0.00364)───╰X──╭C────────────RY(0.0164)──╰X──╭C──────────────RY(-0.0256)──╰X──╭C────────────RY(-0.00445)──╰X──╭C─────────────RY(0.00846)───╰X──╭C─────────────RY(0.0136)───╰X──╭C─────────────RY(0.000727)──╰X──╭C────────────RY(-0.00588)──╰X──╭C───────────RY(0.0159)───╰X──╭C─────────────RY(0.00694)──┤ ⟨Z⟩ 
 2: ──H──RY(0.3)──╭C──╰X─────────────RY(0.000704)──╭C──╰X────────────RY(0.0051)──╭C──╰X──────────────RY(0.00562)──╭C──╰X────────────RY(-0.00411)──╭C──╰X─────────────RY(-0.00722)──╭C──╰X─────────────RY(-0.0168)──╭C──╰X─────────────RY(-0.00155)──╭C──╰X────────────RY(-0.0226)───╭C──╰X────

### Referencias
[1] https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html 

[2] https://pennylane.ai/qml/demos/tutorial_quantum_transfer_learning.html

[3] https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

[4] https://github.com/XanaduAI/quantum-transfer-learning/blob/master/c2q_transfer_learning_cifar.ipynb

[5] https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958

[6] https://arxiv.org/abs/1512.03385